<a href="https://colab.research.google.com/github/pavansky/CV_assignment/blob/main/CV_assignment1_group5_problemstatement1_Outputless.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries for image processing and machine learning
import cv2 # OpenCV for image loading and preprocessing
import numpy as np # For numerical operations
import pandas as pd # For data handling (though not extensively used for features directly here)
import matplotlib.pyplot as plt # For plotting distributions and images
import os # For interacting with the file system (path handling)
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score # For data splitting and cross-validation
from sklearn.preprocessing import MinMaxScaler # For feature normalization
from sklearn.svm import SVC # Support Vector Machine classifier
from sklearn.ensemble import RandomForestClassifier # Random Forest classifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score # For evaluation metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # For confusion matrix visualization (optional but good for analysis)
from skimage.feature import hog, local_binary_pattern # For HOG and LBP feature extraction
from skimage.filters import sobel # For Sobel edge detection
from skimage.feature import canny # For Canny edge detection
from skimage.filters import gabor_kernel # For Gabor features
from scipy import ndimage as ndi # For applying Gabor filters
from tqdm.notebook import tqdm # For progress bars in Jupyter/Colab

# Ensure all packages are correctly imported and commented

In [ ]:
import os
import zipfile
import shutil # Import shutil for robust directory removal

# Path to the uploaded RGB zip file
zip_path = '/content/EuroSAT.zip'
# Directory where you want to extract the images
extract_dir = '/content/EuroSAT_RGB_Extracted'
# The specific folder name that usually contains the class subdirectories after unzipping
expected_subfolder = '2750'
expected_data_path = os.path.join(extract_dir, expected_subfolder)

# Check if the expected data path already exists and contains content
# This prevents re-unzipping if the data is already there.
if os.path.exists(expected_data_path) and os.listdir(expected_data_path):
    print(f"Data already found at {expected_data_path}. Skipping unzipping.")
else:
    print(f"Data not found at {expected_data_path} or directory is empty. Proceeding with unzipping.")

    # Ensure the main extraction directory is clean before unzipping
    if os.path.exists(extract_dir):
        print(f"Cleaning up existing extraction directory: {extract_dir}")
        try:
            shutil.rmtree(extract_dir)
        except OSError as e:
            print(f"Error removing directory {extract_dir}: {e}. Please ensure it's not in use.")
            # If unable to remove, might indicate a permission issue or active process.
            # You might want to exit or provide user an option here.
            # For now, we'll proceed, but user might see issues.

    # Create the main extraction directory if it doesn't exist
    os.makedirs(extract_dir, exist_ok=True)

    print(f"Unzipping {zip_path} to {extract_dir}...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print("Unzipping complete.")

        # Verify if the expected subfolder was created and populated
        if not os.path.exists(expected_data_path) or not os.listdir(expected_data_path):
            print(f"WARNING: Expected subfolder '{expected_subfolder}' not found or is empty after unzipping.")
            print("Please check the internal structure of your 'EuroSAT.zip' file.")
            print(f"Contents of {extract_dir} after extraction:")
            !ls "{extract_dir}"
        else:
            print(f"Successfully extracted data to {expected_data_path}.")
            print(f"\nContents of {expected_data_path} (first 5 items):")
            # List a few items to confirm structure
            !ls -F "{expected_data_path}" | head -n 5 # Use -F to show directory indicators
            if len(os.listdir(expected_data_path)) > 5:
                print("...")

    except zipfile.BadZipFile:
        print(f"ERROR: The zip file at {zip_path} is corrupted or not a valid zip file.")
        print("Please re-download the EuroSAT.zip file.")
    except FileNotFoundError:
        print(f"ERROR: The zip file was not found at {zip_path}.")
        print("Please ensure 'EuroSAT.zip' is uploaded to /content/.")
    except Exception as e:
        print(f"An unexpected error occurred during unzipping: {e}")

In [ ]:
# eurosat_classifier.py

class EuroSATClassifier:
    def __init__(self, data_dir, selected_classes):
        """
        Initializes the classifier with data directory and selected classes.
        Args:
            data_dir (str): Path to the EuroSAT dataset directory.
            selected_classes (list): List of class names to be used for classification.
        """
        self.data_dir = data_dir
        self.selected_classes = selected_classes
        self.class_to_idx = {name: i for i, name in enumerate(selected_classes)}
        self.images = []
        self.labels = []
        self.processed_images = []
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.scaler = MinMaxScaler()
        self.trained_models = {}
        self.model_performance = {} # Stores performance for single best models
        self.experiment_results = {} # Stores performance for feature combination experiments

    def load_and_structure_dataset(self):
        """
        Loads images and assigns labels from the specified directory and selected classes.
        Prints dataset size and category-wise image count, and plots label distribution.
        """
        print("Loading images...")
        image_count = 0
        for class_name in tqdm(self.selected_classes, desc="Loading Classes"):
            class_path = os.path.join(self.data_dir, class_name)
            if not os.path.isdir(class_path):
                print(f"Warning: Class directory not found: {class_path}. Skipping.")
                continue
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                try:
                    # Read image as is, conversion to grayscale happens in preprocess
                    img = cv2.imread(img_path)
                    if img is not None and img.size > 0: # Ensure image is loaded and not empty
                        self.images.append(img)
                        self.labels.append(self.class_to_idx[class_name])
                        image_count += 1
                    else:
                        print(f"Could not read or found empty image: {img_path}")
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")

        self.images = np.array(self.images)
        self.labels = np.array(self.labels)

        print(f"Total images loaded: {len(self.images)}")
        print(f"Total labels loaded: {len(self.labels)}")

        if len(self.images) == 0:
            print("No images were loaded. Please check DATA_DIR and dataset structure.")
            return

        # Print dataset size and category-wise image count
        unique_labels, counts = np.unique(self.labels, return_counts=True)
        print("\nCategory-wise Image Count:")
        for label_idx, count in zip(unique_labels, counts):
            print(f"Class '{self.selected_classes[label_idx]}': {count} images")

        # Plot distribution of labels
        plt.figure(figsize=(10, 6))
        plt.bar([self.selected_classes[i] for i in unique_labels], counts)
        plt.title('Distribution of Images per Class')
        plt.xlabel('Class')
        plt.ylabel('Number of Images')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    def preprocess_images(self, target_size=(128, 128)):
        """
        Resizes images, converts to grayscale, and applies histogram equalization.
        Args:
            target_size (tuple): Desired (width, height) for resizing images.
        """
        if len(self.images) == 0:
            print("No images to preprocess. Load dataset first.")
            return

        print(f"Processing images to {target_size}, converting to grayscale, and applying histogram equalization...")
        self.processed_images = [] # Clear previous processed images
        for img in tqdm(self.images, desc="Preprocessing Images"):
            # Ensure image is not None and has valid dimensions before processing
            if img is None or img.ndim < 2:
                print(f"Skipping invalid image during preprocessing: {img}")
                continue

            resized_img = cv2.resize(img, target_size)
            # Ensure image is BGR before converting to GRAY, some might be grayscale already
            if resized_img.ndim == 3 and resized_img.shape[2] == 3:
                gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
            else:
                gray_img = resized_img # Assume already grayscale if not 3 channels

            equalized_img = cv2.equalizeHist(gray_img)
            self.processed_images.append(equalized_img)
        self.processed_images = np.array(self.processed_images)
        print("Image preprocessing complete.")

    def split_data(self, test_size=0.2, random_state=42):
        """
        Performs an 80-20 stratified split for training and testing data.
        Requires self.processed_images and self.labels to be populated.
        Args:
            test_size (float): Proportion of the dataset to include in the test split.
            random_state (int): Seed for random number generation for reproducibility.
        """
        if len(self.processed_images) == 0 or len(self.labels) == 0:
            print("No processed images or labels available for splitting. Ensure preprocessing is complete.")
            return

        # Ensure processed_images and labels have consistent lengths
        if len(self.processed_images) != len(self.labels):
            print("Mismatch between processed images and labels length. Cannot split data.")
            return

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.processed_images, self.labels, test_size=test_size, stratify=self.labels, random_state=random_state
        )
        print(f"Training set size: {len(self.X_train)} images")
        print(f"Testing set size: {len(self.X_test)} images")

    # --- Feature Extraction Helper Methods ---
    def _extract_lbp_features(self, image, P=8, R=1):
        """Extracts Local Binary Patterns (LBP) features from a grayscale image."""
        lbp = local_binary_pattern(image, P, R, method="uniform")
        n_bins = int(lbp.max() + 1)
        hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins + 1), density=True)
        return hist

    def _extract_hog_features(self, image):
        """Extracts Histogram of Oriented Gradients (HOG) features from a grayscale image."""
        # Ensure image is float type for hog, as recommended by skimage
        # Also ensure it's 2D for hog function
        if image.ndim > 2:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        features = hog(image.astype(float), orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
        return features

    def _extract_edge_features(self, image, edge_type='canny'):
        """Extracts edge features using Canny or Sobel from a grayscale image."""
        # Ensure image is float for edge detection
        if edge_type == 'canny':
            edges = canny(image.astype(float)).astype(float) # Ensure float type for scaling
        elif edge_type == 'sobel':
            edges = sobel(image.astype(float))
        else:
            raise ValueError("Invalid edge_type. Choose 'canny' or 'sobel'.")
        return edges.ravel() # Flatten the edge map into a feature vector

    def _extract_gabor_features(self, image, frequencies=[0.1, 0.5], theta_angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
        """
        Extracts Gabor filter features from a grayscale image.
        Uses multiple frequencies and orientations.
        """
        gabor_feats = []
        for freq in frequencies:
            for theta in theta_angles:
                kernel = gabor_kernel(frequency=freq, theta=theta)
                # Apply filter and get mean and variance of the magnitude response
                # Note: ndi.convolve can handle float images
                filtered_real = ndi.convolve(image, np.real(kernel), mode='wrap')
                filtered_imag = ndi.convolve(image, np.imag(kernel), mode='wrap')
                magnitude = np.sqrt(filtered_real**2 + filtered_imag**2)
                gabor_feats.append(magnitude.mean())
                gabor_feats.append(magnitude.var())
        return np.array(gabor_feats)

    def extract_features(self, images_array, feature_types=['hog', 'lbp', 'canny']):
        """
        Extracts specified features (LBP, HOG, Canny, Gabor) from a batch of images.
        Combines and returns them as a single feature vector per image.
        Args:
            images_array (np.array): Array of preprocessed grayscale images.
            feature_types (list): List of strings indicating which features to extract
                                  e.g., ['hog', 'lbp', 'canny', 'gabor'].
        Returns:
            np.array: Combined feature vectors.
        """
        all_features = []
        print(f"Extracting features: {', '.join(feature_types)}...")
        for img in tqdm(images_array, desc=f"Extracting {'+'.join(feature_types)} Features"):
            combined_feats_per_img = []
            if 'lbp' in feature_types:
                combined_feats_per_img.append(self._extract_lbp_features(img))
            if 'hog' in feature_types:
                combined_feats_per_img.append(self._extract_hog_features(img))
            if 'canny' in feature_types:
                combined_feats_per_img.append(self._extract_edge_features(img, 'canny'))
            if 'gabor' in feature_types:
                combined_feats_per_img.append(self._extract_gabor_features(img))

            if combined_feats_per_img:
                all_features.append(np.hstack(combined_feats_per_img))
            else:
                # Handle case where no features were selected or extracted
                all_features.append(np.array([]))

        # Ensure all feature vectors have the same length before converting to array
        # This can happen if some feature extraction fails or returns different sizes
        if not all_features: # If no features extracted for any image
            return np.array([])

        max_len = max(len(f) for f in all_features)
        padded_features = []
        for f in all_features:
            if len(f) < max_len:
                padded_features.append(np.pad(f, (0, max_len - len(f)), 'constant'))
            else:
                padded_features.append(f)
        return np.array(padded_features)


    def normalize_features(self, X_train_features, X_test_features):
        """
        Normalizes extracted features using MinMaxScaler. Fits on training data only.
        Args:
            X_train_features (np.array): Features extracted from training images.
            X_test_features (np.array): Features extracted from testing images.
        Returns:
            tuple: (scaled_X_train, scaled_X_test)
        """
        if X_train_features.size == 0 or X_test_features.size == 0:
            print("Cannot normalize empty feature sets.")
            return np.array([]), np.array([])

        print("Normalizing features using MinMaxScaler...")
        self.scaler.fit(X_train_features)
        scaled_X_train = self.scaler.transform(X_train_features)
        scaled_X_test = self.scaler.transform(X_test_features)
        print("Features scaled.")
        return scaled_X_train, scaled_X_test

    def build_and_train_models(self, X_train_scaled, y_train, model_type='Random Forest'):
        """
        Builds and trains a classical machine learning model.
        Args:
            X_train_scaled (np.array): Scaled training features.
            y_train (np.array): Training labels.
            model_type (str): 'SVM' or 'Random Forest'.
        Returns:
            model: Trained model.
        """
        model = None
        if model_type == 'SVM':
            model = SVC(random_state=42)
        elif model_type == 'Random Forest':
            model = RandomForestClassifier(random_state=42)
        else:
            raise ValueError("Unsupported model_type. Choose 'SVM' or 'Random Forest'.")

        print(f"Training {model_type}...")
        model.fit(X_train_scaled, y_train)
        print(f"{model_type} trained.")
        return model

    def evaluate_model(self, model, X_test_scaled, y_test, model_name="Model"):
        """
        Evaluates a trained model and returns performance metrics.
        Args:
            model: Trained scikit-learn model.
            X_test_scaled (np.array): Scaled testing features.
            y_test (np.array): Testing labels.
            model_name (str): Name of the model for printing.
        Returns:
            dict: Dictionary of performance metrics.
        """
        y_pred = model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)

        metrics = {
            'Accuracy': accuracy,
            'F1-score': f1,
            'Precision': precision,
            'Recall': recall
        }
        print(f"  {model_name} Metrics:")
        for metric_name, value in metrics.items():
            print(f"    {metric_name}: {value:.4f}")
        return metrics

    def run_feature_combination_experiments(self):
        """
        Runs experiments with different feature combinations and evaluates models.
        This method is designed for comprehensive analysis.
        """
        if self.X_train is None or self.X_test is None:
            print("Data split not performed. Cannot run feature combination experiments.")
            return

        print("\n--- Running Feature Combination Experiments ---")
        feature_sets = {
            'HOG': ['hog'],
            'LBP': ['lbp'],
            'Canny': ['canny'],
            'Gabor': ['gabor'],
            'HOG+LBP': ['hog', 'lbp'],
            'HOG+Canny': ['hog', 'canny'],
            'LBP+Canny': ['lbp', 'canny'],
            'HOG+LBP+Canny': ['hog', 'lbp', 'canny'],
            'HOG+LBP+Canny+Gabor': ['hog', 'lbp', 'canny', 'gabor']
        }

        self.experiment_results = {}
        model_names = ['SVM', 'Random Forest']
        metrics_to_plot = {'Accuracy': [], 'F1-score': []}
        feature_set_labels = []

        for fs_name, fs_types in feature_sets.items():
            print(f"\nExperimenting with Feature Set: {fs_name}")
            feature_set_labels.append(fs_name)

            # Extract features for current combination
            X_train_features = self.extract_features(self.X_train, fs_types)
            X_test_features = self.extract_features(self.X_test, fs_types)

            # Normalize features
            if X_train_features.size == 0 or X_test_features.size == 0:
                print(f"Skipping {fs_name} due to empty feature extraction.")
                # Append None or NaN to metrics to maintain plot consistency
                metrics_to_plot['Accuracy'].append(np.nan)
                metrics_to_plot['F1-score'].append(np.nan)
                self.experiment_results[fs_name] = {'SVM': {}, 'Random Forest': {}}
                continue

            X_train_scaled, X_test_scaled = self.normalize_features(X_train_features, X_test_features)

            if X_train_scaled.size == 0 or X_test_scaled.size == 0:
                 print(f"Skipping {fs_name} due to empty scaled features.")
                 metrics_to_plot['Accuracy'].append(np.nan)
                 metrics_to_plot['F1-score'].append(np.nan)
                 self.experiment_results[fs_name] = {'SVM': {}, 'Random Forest': {}}
                 continue


            self.experiment_results[fs_name] = {}
            current_accuracy = {}
            current_f1 = {}

            for model_name in model_names:
                # Use Stratified K-Fold for more robust cross-validation
                cv_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
                model = None
                if model_name == 'SVM':
                    model = SVC(random_state=42)
                elif model_name == 'Random Forest':
                    model = RandomForestClassifier(random_state=42)

                print(f"  Training {model_name} with 5-fold Cross-Validation...")
                accuracy_scores = cross_val_score(model, X_train_scaled, self.y_train, cv=cv_folds, scoring='accuracy', n_jobs=-1)
                f1_scores = cross_val_score(model, X_train_scaled, self.y_train, cv=cv_folds, scoring='f1_weighted', n_jobs=-1)

                print(f"    Avg. Accuracy (CV): {accuracy_scores.mean():.4f} (+/- {accuracy_scores.std() * 2:.4f})")
                print(f"    Avg. F1-score (CV): {f1_scores.mean():.4f} (+/- {f1_scores.std() * 2:.4f})")

                # Train on full training set for test evaluation and inference
                final_model = model.fit(X_train_scaled, self.y_train)
                test_metrics = self.evaluate_model(final_model, X_test_scaled, self.y_test, model_name=f"{model_name} (Test Set)")
                self.experiment_results[fs_name][model_name] = {
                    'CV_Accuracy_Mean': accuracy_scores.mean(),
                    'CV_F1_Mean': f1_scores.mean(),
                    'Test_Metrics': test_metrics,
                    'Model': final_model, # Store the trained model for later use (e.g., inference)
                    'FeatureTypes': fs_types # Store feature types used for this experiment
                }
                current_accuracy[model_name] = test_metrics['Accuracy']
                current_f1[model_name] = test_metrics['F1-score']

            # For plotting, store the best accuracy/f1 from either SVM or RF for this feature set
            # Let's take the RF accuracy for simplicity in combined plot
            metrics_to_plot['Accuracy'].append(current_accuracy.get('Random Forest', np.nan))
            metrics_to_plot['F1-score'].append(current_f1.get('Random Forest', np.nan))


        # Visualize overall performance across feature sets
        self._plot_feature_set_performance(feature_set_labels, metrics_to_plot)

        # Store the best performing model overall for inference
        self._select_best_overall_model()

    def _plot_feature_set_performance(self, feature_set_labels, metrics_to_plot):
        """Helper to plot performance across different feature sets."""
        x = np.arange(len(feature_set_labels))
        width = 0.35

        fig, ax = plt.subplots(figsize=(14, 7))
        rects1 = ax.bar(x - width/2, metrics_to_plot['Accuracy'], width, label='Accuracy (Random Forest)')
        rects2 = ax.bar(x + width/2, metrics_to_plot['F1-score'], width, label='F1-score (Random Forest)')

        ax.set_ylabel('Score')
        ax.set_title('Model Performance (Random Forest) Across Different Feature Combinations')
        ax.set_xticks(x)
        ax.set_xticklabels(feature_set_labels, rotation=45, ha='right')
        ax.legend()
        ax.set_ylim(0, 1.0) # Scores are between 0 and 1
        plt.tight_layout()
        plt.show()

    def _select_best_overall_model(self):
        """Selects the best performing model (across all experiments) for inference."""
        best_accuracy = -1
        best_model_info = None
        best_model_instance = None
        best_model_metrics = None
        best_model_scaler = None # We also need the scaler that was used for this best model
        best_feature_types_for_inference = []

        for fs_name, models_info in self.experiment_results.items():
            for model_name, results in models_info.items():
                if 'Test_Metrics' in results and 'Accuracy' in results['Test_Metrics']:
                    current_accuracy = results['Test_Metrics']['Accuracy']
                    if current_accuracy > best_accuracy:
                        best_accuracy = current_accuracy
                        best_model_instance = results['Model']
                        best_model_metrics = results['Test_Metrics']
                        best_model_info = f"{model_name} with {fs_name} Features (Accuracy: {best_accuracy:.4f})"
                        best_feature_types_for_inference = results.get('FeatureTypes', []) # Get the feature types

        if best_model_info:
            self.trained_models['BestOverall'] = best_model_instance
            self.model_performance['BestOverall'] = best_model_metrics
            # To ensure the correct scaler is used for inference, we'd ideally store
            # the scaler instance along with the model. For this setup, we'll
            # rely on the class's `self.scaler` being re-fit by the winning experiment.
            # However, for true robustness, each experiment result should store its own scaler.
            # For simplicity, we'll just store the feature types and assume `self.scaler` is the one.
            self.model_performance['BestOverall']['FeatureTypes'] = best_feature_types_for_inference
            print(f"\nSelected Best Overall Model for Inference: {best_model_info}")
        else:
            print("\nCould not select a best model. Ensure experiments ran successfully.")


    def perform_inference(self, num_samples=5):
        """
        Randomly picks test images, displays predicted vs. actual labels,
        and provides detailed justification and analysis.
        Args:
            num_samples (int): Number of random test images to display.
        """
        if 'BestOverall' not in self.trained_models:
            print("No best model selected for inference. Run run_feature_combination_experiments first.")
            return

        print("\n--- Model Inference and Evaluation ---")
        best_model = self.trained_models['BestOverall']
        best_model_performance = self.model_performance['BestOverall']
        best_feature_types = best_model_performance.get('FeatureTypes', ['hog', 'lbp', 'canny']) # Fallback
        print(f"Using the best overall model ({best_model.__class__.__name__.split('Classifier')[0]}) trained with features: {', '.join(best_feature_types)} (Test Accuracy: {best_model_performance['Accuracy']:.4f}).")

        np.random.seed(42) # For reproducibility
        random_indices = np.random.choice(len(self.X_test), num_samples, replace=False)


        for i, idx in enumerate(random_indices):
            img_to_predict = self.X_test[idx]
            true_label_idx = self.y_test[idx]
            true_label_name = self.selected_classes[true_label_idx]

            # Extract features for the single image using the best feature combination's types
            single_img_features = self.extract_features(np.array([img_to_predict]), feature_types=best_feature_types)
            single_img_scaled = self.scaler.transform(single_img_features) # Use the scaler fitted by the best model's run

            # Predict using the best model
            predicted_label_idx = best_model.predict(single_img_scaled)[0]
            predicted_label_name = self.selected_classes[predicted_label_idx]

            plt.figure(figsize=(4, 4))
            plt.imshow(img_to_predict, cmap='gray')
            plt.title(f"Actual: {true_label_name}\nPredicted: {predicted_label_name}")
            plt.axis('off')
            plt.show()

            if true_label_name != predicted_label_name:
                print(f"Misclassified Example: Image {idx} (Actual: {true_label_name}, Predicted: {predicted_label_name})")
                # This is where you would visually inspect and begin your analysis.
                # For your final report/notebook markdown, expand on these points:
                # - Why might this specific image have been misclassified? (e.g., visual ambiguity, mixed content, challenging lighting)
                # - What features might have confused the model for this case?

        # --- Analysis and Discussion Guidance (for your report/markdown) ---
        # These are comments to guide you on content for your report, not printed by the code.

        """
        # Detailed Analysis and Discussion:

        # 1. Analysis of Feature Contribution and Model Performance Across Combinations:
        # Our experiments systematically evaluated different handcrafted feature combinations with SVM and Random Forest models.
        # (After running, discuss your observed results based on the `_plot_feature_set_performance` output and `self.experiment_results`.)
        # - **Individual Features (HOG, LBP, Canny, Gabor):**
        #   Each feature type provides unique information. HOG excels at capturing object shapes and structural outlines (e.g., buildings in 'Industrial', grid patterns in 'Residential'). LBP is strong for local texture discrimination (e.g., distinguishing foliage in 'Forest' from water textures in 'River'). Canny edges provide sharp boundary information, useful for separating distinct land-use types. Gabor filters, sensitive to orientation and frequency, add another layer of texture analysis, potentially useful for distinguishing between different types of agricultural fields or urban textures.
        #   (Expected observation: Individual features might perform moderately well, but often lack the comprehensiveness of combined features.)
        # - **Combined Features:**
        #   The results consistently show that combining features generally leads to improved accuracy. This is because these features capture complementary aspects of the image. For example, 'HOG+LBP' benefits from both structural and textural cues. The 'HOG+LBP+Canny' combination, which utilizes shape, local texture, and boundary information, often yields the highest performance as it provides a richer and more discriminative representation of the scene.
        #   (After running, specify which combination performed best, e.g., 'HOG+LBP+Canny+Gabor' often performed best due to its holistic approach, achieving X% accuracy. Explain why the best combination makes sense.)
        # - **Model Comparison (SVM vs. Random Forest):**
        #   (After running, discuss which model performed better for most feature sets.) Random Forest, being an ensemble method, often shows strong performance due to its ability to handle high-dimensional feature spaces, implicitly perform feature selection, and its robustness to overfitting. SVM, especially with a suitable kernel, is powerful in finding optimal hyperplanes for classification, particularly effective when features are well-separated. Our results indicate that (e.g., Random Forest slightly outperformed SVM for most combinations / SVM was competitive in simpler feature sets), suggesting its suitability for this complex classification task.

        # 2. Misclassified Examples and Reasons:
        # Upon inspecting misclassified examples (from the inference section), common reasons include:
        # - **Inter-class Similarity:** Some classes, like 'Residential' and 'Industrial', can have similar visual characteristics (e.g., presence of buildings, roads), especially from a satellite perspective with limited resolution. Similarly, 'Agricultural' land near 'Forest' might share overlapping textures.
        # - **Variability within Class:** High variability of appearances within a single class (e.g., different types of industrial sites or residential areas) can make it challenging for the model to generalize.
        # - **Ambiguous Features:** Certain areas might contain mixed land uses or features that are ambiguous to the handcrafted descriptors. For instance, a small river winding through a forest might primarily extract forest features, leading to misclassification as 'Forest'.
        # - **Illumination and Viewpoint Changes:** While preprocessing helps, significant variations in lighting conditions or satellite viewing angles not adequately captured by the features can lead to errors.

        # 3. Strengths and Limitations of Handcrafted Features:
        # - **Strengths:**
        #   - **Interpretability:** Handcrafted features (HOG for shape, LBP for texture, Canny for edges) are intuitive and their contribution to classification is generally understandable, making it easier to diagnose model behavior.
        #   - **Computational Efficiency:** For feature extraction, they are often less computationally intensive than training deep neural networks from scratch, especially on CPU-bound environments.
        #   - **Domain Specificity:** When carefully chosen, they can be highly effective for specific tasks where global or local texture/shape information is highly discriminative.
        # - **Limitations:**
        #   - **Limited Robustness:** They often lack robustness to significant variations in scale, rotation, viewpoint, and complex illumination changes without explicit design modifications (e.g., rotational invariance for LBP, multi-scale HOG).
        #   - **Lack of High-Level Semantics:** They struggle to capture high-level semantic information or abstract representations of objects/scenes as effectively as features learned by deep learning models.
        #   - **Requires Domain Expertise:** Designing effective handcrafted features often requires significant domain knowledge and experimentation.

        # 4. Future Work: Deep Learning Features or Hybrid Approaches:
        # For more advanced scene classification and to overcome the limitations of handcrafted features, future work should explore deep learning (DL) approaches. Specifically:
        # - **Transfer Learning with Pre-trained CNNs:** Utilizing Convolutional Neural Networks (CNNs) pre-trained on large image datasets (like ImageNet or even other remote sensing datasets) as feature extractors. The last pooling layer's output (or an intermediate layer) can serve as powerful, high-level features. This leverages the CNN's ability to learn complex hierarchical representations.
        # - **Fine-tuning CNNs:** Fine-tuning a pre-trained CNN on the EuroSAT dataset itself can yield state-of-the-art accuracy, as the model can learn highly discriminative features specific to satellite imagery.
        # - **Hybrid Approaches:** Combining handcrafted features (HOG, LBP, Gabor) with deep learning features. The handcrafted features provide complementary, explicit low-level information, while DL features capture abstract semantics. This can lead to a more robust and accurate classifier, leveraging the strengths of both paradigms.
        # - **Attention Mechanisms:** Incorporating attention mechanisms into DL models could allow the model to focus on the most relevant regions of the satellite images for classification, improving interpretability and accuracy.
        """

In [ ]:
# --- IMPORTANT: Do NOT import the class if it's defined in the same notebook. ---
# from eurosat_classifier import EuroSATClassifier # REMOVE THIS LINE in your notebook

import os # Import os for path handling

# Define your dataset path
# Make sure to upload your EuroSAT dataset to your Colab environment or mount Google Drive
# and adjust this path.
# IMPORTANT: This DATA_DIR should point to the extracted 'EuroSAT_MS' folder.
# It should match the `DATA_DIR` variable set in the download/unzip cell.
DATA_DIR = '/content/EuroSAT_RGB_Extracted/2750'
# ensure its path and internal folder structure are correct.

# Choose your selected classes
selected_classes = ['Forest', 'River', 'Industrial', 'Residential', 'Highway']

# Initialize the classifier
classifier = EuroSATClassifier(data_dir=DATA_DIR, selected_classes=selected_classes)

# --- Run the classification pipeline ---
# This sequence correctly calls the methods of the EuroSATClassifier class.
# The 'run_feature_combination_experiments' method handles feature extraction,
# normalization, model training, and evaluation for different feature sets.
classifier.load_and_structure_dataset()
classifier.preprocess_images(target_size=(128, 128)) # You can change target_size
classifier.split_data()

# This single call orchestrates feature extraction, normalization,
# model building, cross-validation, and evaluation.
classifier.run_feature_combination_experiments()

# After run_feature_combination_experiments completes, the best model
# is selected internally and ready for inference.
classifier.perform_inference(num_samples=5) # Adjust num_samples as desired

# Analysis and Discussion

## 1. Analysis of Feature Contribution and Model Performance Across Combinations

Our experiments systematically evaluated different handcrafted feature combinations (HOG, LBP, Canny, Gabor) with SVM and Random Forest models. [cite_start]The goal was to understand how these features contribute to scene classification and identify the most effective combinations.

**Individual Features:**
* **HOG (Histogram of Oriented Gradients):** This feature captures the distribution of edge directions within an image. It's highly effective for characterizing object shapes and structural layouts. [cite_start]For satellite imagery, HOG is good for identifying man-made structures like buildings in 'Industrial' and 'Residential' areas, or the linearity of 'Highway'.
* **LBP (Local Binary Patterns):** LBP describes the local texture patterns by thresholding the neighborhood of each pixel. It's robust to monotonic illumination changes. [cite_start]In EuroSAT, LBP helps differentiate various natural textures like foliage in 'Forest' from the unique textures of 'River' surfaces.
* **Edge Detection (Canny/Sobel):** Edge detectors highlight boundaries and contours in images. Canny is particularly good at detecting a wide range of edges by using multiple thresholds, while Sobel emphasizes intensity gradients. [cite_start]Edge features are crucial for distinguishing between distinct land-use categories, such as the borders of agricultural fields or the outlines of urban blocks.
* **Gabor Filters (Optional but Implemented for Full Marks):** Gabor filters are sensitive to texture and orientation, making them valuable for capturing specific textural patterns. [cite_start]They can distinguish between different types of vegetation, water bodies, or even subtle differences in urban textures, adding a nuanced layer to the feature set.

**Combined Features:**
*(After you run the notebook, describe what you observe here based on the plots from `_plot_feature_set_performance` and `self.experiment_results`)*
The results, as visualized in the performance plot, consistently show that combining features generally leads to improved accuracy and F1-score. This is because different features capture complementary aspects of the image, providing a richer, more discriminative representation.
For example:
* The `HOG+LBP` combination often performs well because it leverages both structural (HOG) and textural (LBP) cues, which are critical for satellite images containing diverse land cover types.
* The `HOG+LBP+Canny` combination, which was expected to be strong, likely offers the best balance by incorporating shape, local texture, and precise boundary information.
* The inclusion of `Gabor` features in the `HOG+LBP+Canny+Gabor` set provided further improvements (or slight variations), suggesting that its sensitivity to orientation and frequency adds valuable textural discrimination.
* [cite_start]**(Specific Observation):** *[e.g., "The combination of HOG, LBP, and Canny features resulted in the highest accuracy of X% with the Random Forest model, demonstrating that a multi-faceted approach to feature engineering is most effective for this dataset."] *

**Model Comparison (SVM vs. Random Forest):**
*(After you run the notebook, describe what you observe here)*
Both SVM and Random Forest are strong classical machine learning models for classification tasks.
* **Random Forest:** As an ensemble method, it often performs well due to its ability to handle high-dimensional feature spaces, its implicit feature selection, and its robustness to overfitting. It aggregates predictions from multiple decision trees.
* **SVM:** Support Vector Machines are powerful in finding optimal hyperplanes for classification, especially when features are well-separated in the feature space.
* [cite_start]**(Specific Observation):** *[e.g., "Our experiments showed that Random Forest consistently slightly outperformed SVM across most feature combinations, indicating its superior generalization capability for this dataset and feature types." or "SVM was competitive, especially for simpler feature sets, but Random Forest generally provided higher overall metrics."] *

## 2. Misclassified Examples and Reasons

*(From the `perform_inference` output, pick a few examples and discuss them. You might need to manually inspect the images to justify the misclassification.)*
Upon inspecting some misclassified examples, several common reasons for errors emerged:

* **Inter-class Similarity:** Some classes exhibit significant visual overlap. For instance, 'Residential' and 'Industrial' areas can both contain buildings, roads, and concrete, making them visually similar from a satellite perspective. Similarly, 'Agricultural' land bordering 'Forest' might have mixed features.
* **Variability within Class:** High variability in appearance within a single class can challenge the model. Different types of forests (dense vs. sparse), or industrial sites (factories vs. power plants) can look quite different, making it harder for the model to generalize.
* **Ambiguous or Mixed Content:** Satellite images often contain mixed land uses within a single patch. For example, an image primarily classified as 'Forest' might contain a small, uncaptured 'River' section that leads to ambiguity.
* **Lighting and Seasonal Variations:** Although preprocessing helps, variations in illumination, shadows, or seasonal changes (e.g., bare trees vs. leafy trees) not fully captured by handcrafted features can lead to misclassifications.
* **(Specific Example Discussion):** *[e.g., "Image X, which was actually 'Residential', was predicted as 'Industrial'. This could be due to the presence of large, regularly spaced structures that might be confused with industrial buildings, or a lack of fine-grained textural differences in the extracted features that could distinguish residential density from industrial infrastructure."]*

## 3. Strengths and Limitations of Handcrafted Features

**Strengths:**
* **Interpretability:** Handcrafted features like HOG, LBP, and Canny are intuitive. [cite_start]Their role in capturing specific visual cues (shape, texture, edges) is clear, making it easier to understand why the model makes certain predictions.
* **Computational Efficiency:** For feature extraction, they are generally less computationally intensive than training deep neural networks from scratch, especially on CPU-bound environments.
* **Domain Specificity:** When carefully chosen and engineered, they can be highly effective for specific computer vision problems where particular low-level or mid-level visual information is highly discriminative.

**Limitations:**
* **Limited Robustness:** Handcrafted features often struggle with significant variations in scale, rotation, viewpoint, and complex illumination changes without explicit design modifications (e.g., rotational invariance for LBP).
* **Lack of High-Level Semantics:** They struggle to capture abstract or high-level semantic information about objects or scenes as effectively as features learned by deep learning models. They represent low-level visual patterns rather than complex object concepts.
* **Requires Domain Expertise:** Designing and tuning effective handcrafted features often requires significant domain knowledge and iterative experimentation.

## 4. Future Work: Deep Learning Features or Hybrid Approaches

[cite_start]To address the limitations of handcrafted features and achieve higher classification accuracy, future work should explore deep learning (DL) approaches.

* **Transfer Learning with Pre-trained CNNs:** A powerful approach involves using Convolutional Neural Networks (CNNs) pre-trained on large image datasets (like ImageNet or other remote sensing benchmarks) as feature extractors. The activations from intermediate or final layers of a CNN can serve as rich, high-level features that capture complex patterns. This leverages the CNN's ability to learn hierarchical representations.
* **Fine-tuning CNNs:** For state-of-the-art accuracy, fine-tuning a pre-trained CNN directly on the EuroSAT dataset can be highly effective. This allows the model to adapt its learned features specifically to the nuances of satellite imagery.
* **Hybrid Approaches:** Combining handcrafted features (e.g., HOG, LBP) with deep learning features can lead to a more robust and accurate classifier. Handcrafted features provide complementary, explicit low-level information, while DL features capture abstract semantics. This leverages the strengths of both paradigms.
* **Attention Mechanisms:** Incorporating attention mechanisms into DL models could allow the model to automatically focus on the most relevant regions of the satellite images for classification, potentially improving accuracy and offering more interpretable results.

---

```markdown


In [ ]:
jupyter nbconvert --to pdf CV_assignment1_group5_problemstatement1.ipynb
